# FAIIA-IDS Ablation Study (Refactored)

This notebook runs the ablation study for the FAIIA-IDS model by cloning the refactored codebase from GitHub.

**Note:** Please replace `https://github.com/USERNAME/REPO_NAME.git` with your actual repository URL.

In [ ]:
# 1. Clone Repository
# TODO: Replace with your actual repository URL
GIT_REPO_URL = "https://github.com/Arif-Foysal/FAA-Net.git"
REPO_DIR = "FAA-Net" # This usually matches the name of the git repo

!git clone {GIT_REPO_URL}

import os
if os.path.exists(REPO_DIR):
    os.chdir(REPO_DIR)
    print(f"Changed directory to: {os.getcwd()}")
else:
    print(f"Warning: Could not find directory {REPO_DIR}. Check if git clone succeeded.")

# 2. Install Dependencies
!pip install -r requirements.txt

In [ ]:
import sys
# Ensure project root is in path
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

import torch
import pandas as pd
from scripts.run_ablation import main as run_ablation_experiment
from scripts.train_main import main as train_main_model

## 1. Train Main EDAN v3 Model

Trains the full EDAN v3 model with FAIIA and Minority Prototypes using the standard configuration.

In [ ]:
try:
    # Run training logic
    train_main_model()
except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Please ensure dataset files (UNSW_NB15) are present in the project root or /content.")

## 2. Run Ablation Study

Runs 4 experiments:
1. Vanilla DNN + BCE
2. Vanilla DNN + Focal Loss
3. FAIIA + BCE
4. FAIIA + Focal Loss

In [ ]:
try:
    run_ablation_experiment()
except FileNotFoundError as e:
    print(f"Error: {e}")

## 3. View Results

Load and display the summary CSV generated by the ablation study.

In [ ]:
if os.path.exists('ablation_summary.csv'):
    df = pd.read_csv('ablation_summary.csv', index_col=0)
    display(df)
else:
    print("Results file not found. Ensure ablation study ran successfully.")